# 게임 데이터 전처리
### DB에 삽입하기 위한 데이터 전처리
1. 전체 데이터 합하기
2. 필요없는 행 제거
    - 필수 데이터가 없는 경우 (유료 게임은 제외해서 받았기 때문에 데이터가 없다면 모두 제거)
3. 모든 열에 대해서 결측치 확인
    - price의 결측값 모두 0 삽입 (무료니까)
    - developers의 결측값 모두 '-' 삽입 (게임을 플레이할 수 없는 건 아니기 때문에)
    - recommendations의 결측값 모두 0 삽입
    - metacritics 결측값 모두 0 삽입
4. 출시예정게임 삭제
5. categories, genres, developers의 타입이 list -> str
6. price 00 떼기

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')

---

## 유료 데이터 합하기

In [2]:
# 유료 데이터 합하기
game_data = pd.read_excel('data/game_1_data.xlsx')
for i in range(2, 17):
    temp_data = pd.read_excel(f'data/game_{i}_data.xlsx')
    game_data = pd.concat([game_data, temp_data])
    
# 전체 개수 : 158,034

In [3]:
# dlc인 게임 삭제 
game_data = game_data[game_data['type']=='game']

# 개수 : 57,068

In [4]:
# type 날리기
game_data = game_data.drop('type', axis=1)

In [5]:
game_data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
4,1610530,The Annoying Game,"Run, jump, die, repeat. Help Teddy on his jour...",440000.0,['싱글 플레이어'],"['캐주얼', '인디']",NaN,2021년 7월 14일,"['Michal Ambrozej', 'The Piano Chap']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,'The Annoying Game' is a retro style platforme...,['https://cdn.akamai.steamstatic.com/steam/app...
10,1610610,CallMeProducer101,"作为曾经的电视台天才制作人,你要担负起经营一个全新视频网站的责任 节目制作，节目安排，营销规...",220000.0,"['싱글 플레이어', 'Steam Cloud']","['시뮬레이션', '전략']",NaN,2021년 5월 18일,['随便做做游戏组'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"中国内地娱乐视频网站模拟器<br />_x000D_\n作为一个曾经的电视台天才制作人,因为...",['https://cdn.akamai.steamstatic.com/steam/app...
11,1610630,BURGER MAN,&quot;BURGER MAN&quot; is a hardcore 2D platfo...,110000.0,"['싱글 플레이어', 'Steam 도전 과제']","['캐주얼', '인디']",NaN,2021년 5월 6일,['Rex Junior'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,&quot;BURGER MAN&quot; is a hardcore side-scro...,['https://cdn.akamai.steamstatic.com/steam/app...
13,1610650,●LIVE IN DUNGEON,●LIVE IN DUNGEON is an RPG where you play as a...,1050000.0,['싱글 플레이어'],"['인디', 'RPG']",NaN,2021년 6월 4일,['Heaviside Creations'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"Wonderlucia, an alternate world where streamin...",['https://cdn.akamai.steamstatic.com/steam/app...
14,1610660,Throw Knives,Throw Knives is a new casual and addictive ret...,1050000.0,['싱글 플레이어'],"['캐주얼', '인디', '시뮬레이션', '전략']",NaN,2021년 5월 6일,['Stas Paty9mkin'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,<i> <strong> Throw Knives </strong> </i> is a ...,['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,537000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['캐주얼', '인디', 'RPG', '전략']",318.0,2020년 3월 13일,['Wonderbelly Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9864,548430,Deep Rock Galactic,"딥 락 갤럭틱은 1-4인의 플레이어가 협동하는 FPS 게임으로, 상남자 우주 드워프...",1023000.0,"['싱글 플레이어', '멀티플레이어', '협동', '온라인 협동', 'Steam 도...",['액션'],155214.0,2020년 5월 13일,['Ghost Ship Games'],85.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9870,2196390,2022生存指南 2022 SURVIVAL GUIDE,1.Survival plot puzzle video game 2.Linear gam...,204000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2023년 3월 14일,"['塔克设计工作室DARK STUDIO', '毛玮欣MAO WEIXIN']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,2022 Survival Guide is a survival action puzzl...,['https://cdn.akamai.steamstatic.com/steam/app...
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,550000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['액션', '캐주얼', '인디', 'RPG']",NaN,2022년 7월 23일,['Cosmos'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,['https://cdn.akamai.steamstatic.com/steam/app...


## 무료 데이터 합하기

In [6]:
# 전체 데이터 합하기
free_game_data = pd.read_excel('data/free/game_free_1_data.xlsx')
for i in range(2, 12):
    temp_data = pd.read_excel(f'data/free/game_free_{i}_data.xlsx')
    free_game_data = pd.concat([free_game_data, temp_data])

In [7]:
free_game_data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
0,216938,Pieterw test app76 ( 216938 ),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,660010,test2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,660130,test3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1610540,Train Station Renovation - Germany DLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9173,2338830,Rocket Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9174,1062810,Inkbound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9175,1185980,Pathfinder: Wrath of the Righteous Test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9176,774941,Squad - Public Testing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 전체 데이터 합하기

In [8]:
data = pd.concat([free_game_data, game_data])

In [9]:
# 개수 : 158,034 
data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
0,216938,Pieterw test app76 ( 216938 ),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,660010,test2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,660130,test3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1610540,Train Station Renovation - Germany DLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,537000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['캐주얼', '인디', 'RPG', '전략']",318.0,2020년 3월 13일,['Wonderbelly Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9864,548430,Deep Rock Galactic,"딥 락 갤럭틱은 1-4인의 플레이어가 협동하는 FPS 게임으로, 상남자 우주 드워프...",1023000.0,"['싱글 플레이어', '멀티플레이어', '협동', '온라인 협동', 'Steam 도...",['액션'],155214.0,2020년 5월 13일,['Ghost Ship Games'],85.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9870,2196390,2022生存指南 2022 SURVIVAL GUIDE,1.Survival plot puzzle video game 2.Linear gam...,204000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2023년 3월 14일,"['塔克设计工作室DARK STUDIO', '毛玮欣MAO WEIXIN']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,2022 Survival Guide is a survival action puzzl...,['https://cdn.akamai.steamstatic.com/steam/app...
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,550000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['액션', '캐주얼', '인디', 'RPG']",NaN,2022년 7월 23일,['Cosmos'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,['https://cdn.akamai.steamstatic.com/steam/app...


---

# 필요없는 데이터 삭제

In [10]:
# 없는 데이터 삭제
# 헤더 이미지가 없는 게임은 없기에 이를 기준으로 !!
data.dropna(subset=['image'], inplace=True)

---

# 결측치 확인하기

In [11]:
data[data['short_description'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
7181,1654040,Overdosed,NaN,NaN,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원']","['액션', '어드벤처', '캐주얼', '시뮬레이션']",NaN,NaN,['CapitalGamingRP Studios'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
2222,1273470,Isle Clash,NaN,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP', '협동', ...","['캐주얼', '무료', '인디', '대규모 멀티플레이어', 'RPG', '전략']",NaN,출시 예정 게임,['EnCoStudio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
4554,1295030,Zagan Must Be Rescued,NaN,NaN,"['싱글 플레이어', '컨트롤러 일부 지원']","['액션', '캐주얼', '인디']",NaN,2020년 5월 11일,"['Selim Akyol', 'Yusuf Akyol']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Zagan Must Be Rescued is a FPS survival game w...,['https://cdn.akamai.steamstatic.com/steam/app...
3000,1467600,Zinuru The Great,NaN,110000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2020년 12월 2일,['GRC'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
4042,523080,Footbrawl Playground,NaN,650000.0,"['싱글 플레이어', '멀티플레이어', 'PvP', '스크린 공유 및 분할 PvP'...","['액션', '캐주얼', '인디', '스포츠', '앞서 해보기']",NaN,2016년 9월 1일,['K'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
1904,1176440,Math The Question,NaN,110000.0,['싱글 플레이어'],"['캐주얼', '인디']",NaN,2019년 10월 24일,"[""K'sG""]",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,['https://cdn.akamai.steamstatic.com/steam/app...
2258,1171870,魂之大陆 Soultia,NaN,650000.0,['싱글 플레이어'],"['액션', '어드벤처', '캐주얼', '인디', 'RPG', '앞서 해보기']",NaN,2019년 11월 1일,['Pisces Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,<strong> &quot;Soultia&quot; </strong> is a on...,['https://cdn.akamai.steamstatic.com/steam/app...
5798,1099920,Pro Strategy Football 2020,NaN,1250000.0,"['싱글 플레이어', '멀티플레이어', '공유/분할 화면', '레벨 에디터 포함',...","['인디', '시뮬레이션', '스포츠', '전략']",NaN,2019년 9월 13일,['Kerry Batts'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,<br><br>Welcome to the 2020 evolution of Pro ...,['https://cdn.akamai.steamstatic.com/steam/app...


In [12]:
data[data['price'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,NaN,['싱글 플레이어'],"['액션', '캐주얼', '무료', '인디', '전략']",NaN,2021년 5월 24일,['DropOuts'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,['https://cdn.akamai.steamstatic.com/steam/app...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['액션', '앞서 해보기']",NaN,2021년 5월 30일,['Production Games Inc'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,['https://cdn.akamai.steamstatic.com/steam/app...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['캐주얼', '시뮬레이션', '앞서 해보기']",NaN,2021년 10월 11일,['Tigerlilly2931'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,['https://cdn.akamai.steamstatic.com/steam/app...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,NaN,['싱글 플레이어'],"['액션', '인디', 'RPG']",NaN,2021년 5월 23일,['Epic Banana Games LLC'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,['https://cdn.akamai.steamstatic.com/steam/app...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '어드벤처', '무료', '대규모 멀티플레이어', 'RPG', '시뮬레...",NaN,2021년 5월 9일,['Tactigon Games Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,1380410,Puzzle Quest 3,세계적인 퍼즐 RPG 프랜차이즈의 새 시리즈 퍼즐 퀘스트 3(Puzzle Quest...,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP', '협동', ...","['액션', '어드벤처', '캐주얼', '무료', 'RPG', '전략', '앞서 해...",NaN,2022년 2월 28일,['Infinity Plus 2'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,세계적인 퍼즐 RPG 프랜차이즈의 새 시리즈 퍼즐 퀘스트 3(Puzzle Quest...,['https://cdn.akamai.steamstatic.com/steam/app...
9155,1506600,My boss is weird,&quot;My boss is weird&quot; is a career AVG c...,NaN,"['싱글 플레이어', 'Steam 도전 과제', 'Steam Cloud']","['캐주얼', '무료', '인디']",NaN,2021년 2월 2일,['oldwang'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"Enter the <strong> <a href=""https://steamcommu...",['https://cdn.akamai.steamstatic.com/steam/app...
9158,2250690,Sky Strikers VR,Sky Strikers merges sports excitement with imm...,NaN,"['멀티플레이어', 'PvP', '온라인 PvP', '협동', '온라인 협동', '...","['캐주얼', '무료', '인디', '스포츠', '앞서 해보기']",NaN,2023년 3월 5일,['CHAMBER 8 Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Sky Strikers combines the thrill of sports gam...,['https://cdn.akamai.steamstatic.com/steam/app...
9165,835570,Conqueror's Blade,컨커러스 블레이드에서는 누구든지 위대한 장군이 될 수 있습니다. 부대를 이끌고 전 ...,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '무료', '대규모 멀티플레이어', 'RPG', '시뮬레이션', '전략']",3826.0,2020년 4월 5일,['Booming Tech'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...


In [13]:
data[data['categories'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots


In [14]:
data[data['genres'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots


In [15]:
data[data['recommendations'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,NaN,['싱글 플레이어'],"['액션', '캐주얼', '무료', '인디', '전략']",NaN,2021년 5월 24일,['DropOuts'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,['https://cdn.akamai.steamstatic.com/steam/app...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['액션', '앞서 해보기']",NaN,2021년 5월 30일,['Production Games Inc'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,['https://cdn.akamai.steamstatic.com/steam/app...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['캐주얼', '시뮬레이션', '앞서 해보기']",NaN,2021년 10월 11일,['Tigerlilly2931'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,['https://cdn.akamai.steamstatic.com/steam/app...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,NaN,['싱글 플레이어'],"['액션', '인디', 'RPG']",NaN,2021년 5월 23일,['Epic Banana Games LLC'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,['https://cdn.akamai.steamstatic.com/steam/app...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '어드벤처', '무료', '대규모 멀티플레이어', 'RPG', '시뮬레...",NaN,2021년 5월 9일,['Tactigon Games Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9853,1549440,MeiQi 2019,The &quot;Meiqi&quot; series is a plot-oriente...,440000.0,"['싱글 플레이어', 'Steam 도전 과제', 'Steam Cloud']","['인디', '앞서 해보기']",NaN,2022년 11월 3일,['QZQ Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<h2 class=""bb_tag"">MeiQi Series Introduction</...",['https://cdn.akamai.steamstatic.com/steam/app...
9855,2333720,Draft Day Sports: College Basketball 2023,Draft Day Sports: College Basketball 2023 puts...,4200000.0,['싱글 플레이어'],"['인디', '시뮬레이션', '스포츠', '전략']",NaN,2023년 3월 10일,['Wolverine Studios'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Draft Day Sports: College Basketball 2023 lets...,['https://cdn.akamai.steamstatic.com/steam/app...
9856,1366560,MiLE HiGH TAXi,'The Fifth Element' Meets 'Crazy Taxi'! Delive...,1572000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['액션', '인디', '레이싱', '시뮬레이션']",NaN,2023년 3월 13일,['Cassius John-Adams'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<h2 class=""bb_tag"">A High-Octane Thrill Ride</...",['https://cdn.akamai.steamstatic.com/steam/app...
9870,2196390,2022生存指南 2022 SURVIVAL GUIDE,1.Survival plot puzzle video game 2.Linear gam...,204000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2023년 3월 14일,"['塔克设计工作室DARK STUDIO', '毛玮欣MAO WEIXIN']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,2022 Survival Guide is a survival action puzzl...,['https://cdn.akamai.steamstatic.com/steam/app...


In [16]:
data[data['release_date'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
2705,1411750,Your Fear,You must find your daughter and wife in a hosp...,NaN,['싱글 플레이어'],"['어드벤처', '인디']",NaN,NaN,['Gergo Blasko'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<h2 class=""bb_tag"">ATTENTION: PROJECT IS ON HO...",['https://cdn.akamai.steamstatic.com/steam/app...
2994,1406030,Powamo,Have Fun fighting with your Powamo in the Fant...,NaN,"['멀티플레이어', 'PvP', '온라인 PvP', '앱 내 구매', '컨트롤러 일...","['액션', '어드벤처', '캐주얼', '무료']",NaN,NaN,['Altumedia'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Powamo is a 3rd person Battle Royale with Crea...,['https://cdn.akamai.steamstatic.com/steam/app...
3262,1399900,PolyBeyond,Welcome to the PolyBeyond Corporation. In this...,NaN,"['멀티플레이어', 'PvP', '온라인 PvP', 'Steam 도전 과제', '앱...","['액션', '어드벤처', '인디', '대규모 멀티플레이어']",NaN,NaN,['UnityGuy'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,PolyBeyond is a Third Person Multiplayer Loot ...,['https://cdn.akamai.steamstatic.com/steam/app...
7,1512380,Dead Zone,Gathering Storm Studio proudly present Dead Zo...,NaN,"['싱글 플레이어', 'Steam 도전 과제', '앱 내 구매']","['액션', '무료', '인디']",NaN,NaN,['Gathering Storm Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"Set in the grim near future, humanity has been...",['https://cdn.akamai.steamstatic.com/steam/app...
113,1508500,Last Hour,Survive! - Fight! - Be careful!,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP', '협동', ...","['액션', '무료', '인디']",NaN,NaN,"['Pascal Hercher', 'Tom Thoß']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,It starts with a relaxed cruise. But it doesn'...,['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4696,242050,Assassin's Creed IV Black Flag,The year is 1715. Pirates rule the Caribbean a...,4400000.0,"['싱글 플레이어', '멀티플레이어', '앱 내 구매']","['액션', '어드벤처']",50236.0,NaN,['Ubisoft Montreal'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,The year is 1715. Pirates rule the Caribbean a...,['https://cdn.akamai.steamstatic.com/steam/app...
5097,233530,Wake,Wake is the story of an engineer trapped on a ...,550000.0,"['싱글 플레이어', 'Steam 트레이딩 카드', '컨트롤러 일부 지원']","['액션', '인디']",115.0,NaN,['Boss Baddie'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Wake is the story of an engineer trapped on a ...,['https://cdn.akamai.steamstatic.com/steam/app...
5573,223850,3DMark,"3DMark는 하드웨어의 성능을 더욱 더 활용하고자 하는 게이머, 오버클럭 사용자,...",3600000.0,"['Steam 도전 과제', 'Steam 트레이딩 카드', 'Steam 순위표']",['유틸리티'],11119.0,NaN,['UL Solutions'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"PC 게임은 더 빠른 프레임 속도, 더 높은 해상도 및 더 나은 품질의 시각적 효과...",['https://cdn.akamai.steamstatic.com/steam/app...
3815,8980,Borderlands GOTY,Get ready for the mind blowing insanity! Play ...,3170000.0,"['싱글 플레이어', '멀티플레이어', '협동', 'Steam 도전 과제', '컨트...","['액션', 'RPG']",18850.0,NaN,['Gearbox Software'],81.0,https://cdn.akamai.steamstatic.com/steam/apps/...,광기에 대면할 준비를 하십시오.<br> 정신 나간 행위를 벌일 준비...,['https://cdn.akamai.steamstatic.com/steam/app...


In [17]:
data[data['developers'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
7773,1469220,Cat Simulator,"If your life is tired, just become a cute and ...",NaN,['싱글 플레이어'],"['캐주얼', 'RPG', '시뮬레이션', '앞서 해보기']",NaN,2021년 6월 8일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"If your life is tired, just be a cat, become a...",['https://cdn.akamai.steamstatic.com/steam/app...
6600,1763330,Polyslime,An action survival game where you will craft w...,NaN,"['싱글 플레이어', '멀티플레이어', '협동', '온라인 협동']","['액션', '전략']",NaN,2021년 10월 13일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,In this game the goal is simply to survive as ...,['https://cdn.akamai.steamstatic.com/steam/app...
5218,776140,Idol Quest VR,"달리기, 점프, 수영, 등반, VR에서 쏴라. 위험한 동물이나 다른 위험을 피하면서...",NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['폭력적', '고어', '액션', '어드벤처', '인디']",NaN,2018년 3월 26일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Idol Quest VR은 잃어버린 우상을 발견하는 간단한 전제가있는 싱글 플레이어...,['https://cdn.akamai.steamstatic.com/steam/app...
7782,688150,JUMPER : SPEEDRUN,JUMPER : SPEEDRUN is a new Type of FPS Platfor...,NaN,['싱글 플레이어'],"['인디', '레이싱']",NaN,2017년 8월 28일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,<strong>JUMPER : SPEEDRUN</strong> is an FPS P...,['https://cdn.akamai.steamstatic.com/steam/app...
5747,1240910,神鬼降世,《神鬼降世》是一款即时战斗类MMORPG网页游戏游戏 游戏整体上延用了经典的战士、法师、牧师...,NaN,"['싱글 플레이어', '멀티플레이어', 'MMO', '앱 내 구매']","['액션', '무료', '인디', '대규모 멀티플레이어', 'RPG']",NaN,2020년 2월 18일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,《神鬼降世》是一款即时战斗类MMORPG网页游戏游戏<br />\r\n游戏整体上延用了经典...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4887,237370,PlayClaw 5,The amazing PlayClaw video capture tool gives ...,4300000.0,['Steam 도전 과제'],"['오디오 제작', '소프트웨어 교육', '유틸리티', '동영상 제작']",353.0,2013년 9월 10일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,The amazing PlayClaw video capture tool gives ...,['https://cdn.akamai.steamstatic.com/steam/app...
5405,227020,Rise of Venice,"Venice: As a young man striving for success, p...",2360000.0,"['싱글 플레이어', '멀티플레이어', 'Steam 도전 과제', 'Steam 트레...","['시뮬레이션', '전략']",303.0,2013년 9월 27일,NaN,66.0,https://cdn.akamai.steamstatic.com/steam/apps/...,Venice was at the peak of its power during the...,['https://cdn.akamai.steamstatic.com/steam/app...
6346,379260,Atlantic Quest Solitaire,It's &quot;High Fins and cards galore!&quot; a...,950000.0,['싱글 플레이어'],['캐주얼'],NaN,2015년 6월 22일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Clowny and Sharky never seem to find a single ...,['https://cdn.akamai.steamstatic.com/steam/app...
8271,355150,gravilon,"An inventive, thought-provoking puzzle platfor...",330000.0,"['싱글 플레이어', 'Steam 트레이딩 카드', '컨트롤러 일부 지원']","['어드벤처', '인디', '전략']",1104.0,2015년 3월 30일,NaN,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<strong>Gravilon</strong> is an inventive, tho...",['https://cdn.akamai.steamstatic.com/steam/app...


In [18]:
data[data['metacritic'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,NaN,['싱글 플레이어'],"['액션', '캐주얼', '무료', '인디', '전략']",NaN,2021년 5월 24일,['DropOuts'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,['https://cdn.akamai.steamstatic.com/steam/app...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['액션', '앞서 해보기']",NaN,2021년 5월 30일,['Production Games Inc'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,['https://cdn.akamai.steamstatic.com/steam/app...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",NaN,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['캐주얼', '시뮬레이션', '앞서 해보기']",NaN,2021년 10월 11일,['Tigerlilly2931'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,['https://cdn.akamai.steamstatic.com/steam/app...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,NaN,['싱글 플레이어'],"['액션', '인디', 'RPG']",NaN,2021년 5월 23일,['Epic Banana Games LLC'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,['https://cdn.akamai.steamstatic.com/steam/app...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '어드벤처', '무료', '대규모 멀티플레이어', 'RPG', '시뮬레...",NaN,2021년 5월 9일,['Tactigon Games Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,719890,Beasts of Bermuda,Beasts of Bermuda is a multiplayer game hostin...,2050000.0,"['멀티플레이어', 'PvP', '온라인 PvP']","['어드벤처', '인디', '대규모 멀티플레이어', '시뮬레이션', '앞서 해보기']",5490.0,2018년 12월 21일,"['Sastrei Studios, LLC']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,537000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['캐주얼', '인디', 'RPG', '전략']",318.0,2020년 3월 13일,['Wonderbelly Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9870,2196390,2022生存指南 2022 SURVIVAL GUIDE,1.Survival plot puzzle video game 2.Linear gam...,204000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2023년 3월 14일,"['塔克设计工作室DARK STUDIO', '毛玮欣MAO WEIXIN']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,2022 Survival Guide is a survival action puzzl...,['https://cdn.akamai.steamstatic.com/steam/app...
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,550000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['액션', '캐주얼', '인디', 'RPG']",NaN,2022년 7월 23일,['Cosmos'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,['https://cdn.akamai.steamstatic.com/steam/app...


In [19]:
data[data['about_the_game'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
7181,1654040,Overdosed,NaN,NaN,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원']","['액션', '어드벤처', '캐주얼', '시뮬레이션']",NaN,NaN,['CapitalGamingRP Studios'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
1224,725870,Age of seas,Space Thieves embarks you on a stellar mission...,NaN,"['싱글 플레이어', '멀티플레이어', 'MMO', 'PvP', '온라인 PvP',...","['액션', '어드벤처', '인디', '대규모 멀티플레이어', '시뮬레이션']",NaN,출시 예정 게임,['Vector bundle'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
2222,1273470,Isle Clash,NaN,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP', '협동', ...","['캐주얼', '무료', '인디', '대규모 멀티플레이어', 'RPG', '전략']",NaN,출시 예정 게임,['EnCoStudio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
608,1601260,Project Scav,Project Scav is a Low Poly Parody of every Har...,1550000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', '시뮬레이션', '앞서 해보기']",NaN,2021년 9월 30일,"['Cannabis', 'Meta']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
3000,1467600,Zinuru The Great,NaN,110000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",NaN,2020년 12월 2일,['GRC'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
3942,1622990,Dr. Byte,닥터 바이트는 바이러스로 가득 찬 컴퓨터 시스템의 깊이에서 서사적 인 모험입니다. ...,210000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디']",NaN,2021년 6월 21일,['RadLab'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,['https://cdn.akamai.steamstatic.com/steam/app...
4042,523080,Footbrawl Playground,NaN,650000.0,"['싱글 플레이어', '멀티플레이어', 'PvP', '스크린 공유 및 분할 PvP'...","['액션', '캐주얼', '인디', '스포츠', '앞서 해보기']",NaN,2016년 9월 1일,['K'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
1904,1176440,Math The Question,NaN,110000.0,['싱글 플레이어'],"['캐주얼', '인디']",NaN,2019년 10월 24일,"[""K'sG""]",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,['https://cdn.akamai.steamstatic.com/steam/app...


In [20]:
data[data['screenshots'].isnull()]

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
3393,1396170,Paint Royale,Paint Royale is a Paintball Co-op shooter in w...,NaN,"['멀티플레이어', 'PvP', '온라인 PvP', '플랫폼간 멀티플레이어', '컨...","['액션', '무료', '인디', '앞서 해보기']",NaN,출시 예정 게임,"['Epic Nova Studios', 'Bad Alien Games']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Paint Royale is a paintball shooter that goes ...,NaN
5608,1372870,ゴリラの奇跡 ~ Miracle of Gorilla,d,NaN,"['싱글 플레이어', 'Steam 도전 과제']","['액션', '어드벤처', '무료']",NaN,2026년 12월 7일,['d'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,d,NaN
7181,1654040,Overdosed,NaN,NaN,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원']","['액션', '어드벤처', '캐주얼', '시뮬레이션']",NaN,NaN,['CapitalGamingRP Studios'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
1833,1081470,Born of Fire,The free trial version of Born of Fire include...,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP']","['무료', 'RPG', '전략', '앞서 해보기']",NaN,2019년 9월 4일,['Leaf Boat Studio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,The free trial version of Born of Fire include...,NaN
5837,797660,Legend Knight,God Test,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '대규모 멀티플레이어', 'RPG', '전략']",NaN,2018년 4월 18일,['God Test'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,God Test,NaN
1224,725870,Age of seas,Space Thieves embarks you on a stellar mission...,NaN,"['싱글 플레이어', '멀티플레이어', 'MMO', 'PvP', '온라인 PvP',...","['액션', '어드벤처', '인디', '대규모 멀티플레이어', '시뮬레이션']",NaN,출시 예정 게임,['Vector bundle'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN
4357,959220,Country Desert Simulator,Project discontinued,NaN,"['싱글 플레이어', 'Steam 도전 과제', 'Steam 창작마당', 'Stea...","['액션', '어드벤처', '캐주얼', '무료', '시뮬레이션', '전략', '앞서...",NaN,출시 예정 게임,['Benjamin Zekavica'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,Project discontinued,NaN
6930,855830,NEX:OS,NEX:OS is a community driven Ability/Movement ...,NaN,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '컨트롤...","['액션', '어드벤처', '무료', '인디', 'RPG', '앞서 해보기']",NaN,출시 예정 게임,['Primordial Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NEX:OS is a community driven Ability/Movement ...,NaN
943,1850200,ScreamBox,"This game is no longer in development, we do n...",NaN,"['싱글 플레이어', 'Steam 도전 과제', '통계', 'Steam 순위표']","['액션', '어드벤처', '캐주얼', '무료', '인디', '전략']",NaN,출시 예정 게임,['Luke Matheson'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,"This game is no longer in development, we do n...",NaN
2222,1273470,Isle Clash,NaN,NaN,"['싱글 플레이어', '멀티플레이어', 'PvP', '온라인 PvP', '협동', ...","['캐주얼', '무료', '인디', '대규모 멀티플레이어', 'RPG', '전략']",NaN,출시 예정 게임,['EnCoStudio'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN


### 결측치 채우기

In [21]:
# recommendations에 0 넣기
data['recommendations'] = data['recommendations'].fillna(0)

In [22]:
# metacritic에 0 넣기
data['metacritic'] = data['metacritic'].fillna(0)

In [23]:
# price에 0 넣기
data['price'] = data['price'].fillna(0)

In [24]:
# developers에 '-' 넣기
data['developers'] = data['developers'].fillna('-')

---

# 출시예정게임 삭제

In [25]:
# 출시 예정 게임 삭제
# ★ "coming_soon": true를 처음에 체크할 것
# '발표 예정', '출시 예정 게임', '2024년', '2023년 3분기', '2023년', nan, '2023년 2분기'

# (1) 결측치 제거
data = data.dropna(subset=['release_date'])

# (2) 정규표현식을 사용하여 출시 예정인 게임 제거
pattern = r'^\d{4}년 \d{1,2}월 \d{1,2}일$'

# 정규표현식으로 필터링하여 새로운 데이터프레임 생성
data = data[data['release_date'].str.match(pattern)]

In [26]:
data['release_date'] = pd.to_datetime(data['release_date'], format='%Y년 %m월 %d일')

In [27]:
data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,0.0,['싱글 플레이어'],"['액션', '캐주얼', '무료', '인디', '전략']",0.0,2021-05-24,['DropOuts'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,['https://cdn.akamai.steamstatic.com/steam/app...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,0.0,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['액션', '앞서 해보기']",0.0,2021-05-30,['Production Games Inc'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,['https://cdn.akamai.steamstatic.com/steam/app...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",0.0,"['싱글 플레이어', '추적되는 컨트롤러 지원', 'VR 전용']","['캐주얼', '시뮬레이션', '앞서 해보기']",0.0,2021-10-11,['Tigerlilly2931'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,['https://cdn.akamai.steamstatic.com/steam/app...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,0.0,['싱글 플레이어'],"['액션', '인디', 'RPG']",0.0,2021-05-23,['Epic Banana Games LLC'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,['https://cdn.akamai.steamstatic.com/steam/app...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,0.0,"['멀티플레이어', 'MMO', 'PvP', '온라인 PvP', '협동', '온라인...","['액션', '어드벤처', '무료', '대규모 멀티플레이어', 'RPG', '시뮬레...",0.0,2021-05-09,['Tactigon Games Studio'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,537000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['캐주얼', '인디', 'RPG', '전략']",318.0,2020-03-13,['Wonderbelly Games'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9864,548430,Deep Rock Galactic,"딥 락 갤럭틱은 1-4인의 플레이어가 협동하는 FPS 게임으로, 상남자 우주 드워프...",1023000.0,"['싱글 플레이어', '멀티플레이어', '협동', '온라인 협동', 'Steam 도...",['액션'],155214.0,2020-05-13,['Ghost Ship Games'],85.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",['https://cdn.akamai.steamstatic.com/steam/app...
9870,2196390,2022生存指南 2022 SURVIVAL GUIDE,1.Survival plot puzzle video game 2.Linear gam...,204000.0,['싱글 플레이어'],"['액션', '어드벤처', '인디', 'RPG']",0.0,2023-03-14,"['塔克设计工作室DARK STUDIO', '毛玮欣MAO WEIXIN']",0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,2022 Survival Guide is a survival action puzzl...,['https://cdn.akamai.steamstatic.com/steam/app...
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,550000.0,"['싱글 플레이어', 'Steam 도전 과제', '컨트롤러 완벽 지원', 'Stea...","['액션', '캐주얼', '인디', 'RPG']",0.0,2022-07-23,['Cosmos'],0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,['https://cdn.akamai.steamstatic.com/steam/app...


In [28]:
mask = data['release_date'] <= '2023-03-13'

In [29]:
data = data.loc[mask]

In [30]:
# data['release_date'] = data['release_date'].dt.strftime('%Y년 %m월 %d일')

---

# categories, genres, developers, screenshots 형변환

In [31]:
# categories, genres, screenshots 타입이 list인 거 str으로. (따옴표는 제거, 쉼표로 구분)
def list_to_str(list_item):
    if(pd.isnull(list_item)):
        return list_item
    return ''.join(list_item).replace('[','').replace(']','').replace("'", '').replace(", ", ',')

In [32]:
# developers의 경우 쉼표를 포함하는 개발사 이름이 있기 때문에 다르게 적용. (구분자 : ///)
def list_to_str2(list_item):
    if(pd.isnull(list_item)):
        return list_item
    return list_item.strip('[]').replace("', '",'///').strip("''")

In [33]:
data['categories'] = data['categories'].apply(list_to_str)
data['genres'] = data['genres'].apply(list_to_str)
data['developers'] = data['developers'].apply(list_to_str2)
data['screenshots'] = data['screenshots'].apply(list_to_str)

In [34]:
data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,0.0,싱글 플레이어,"액션,캐주얼,무료,인디,전략",0.0,2021-05-24,DropOuts,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,https://cdn.akamai.steamstatic.com/steam/apps/...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,0.0,"싱글 플레이어,추적되는 컨트롤러 지원,VR 전용","액션,앞서 해보기",0.0,2021-05-30,Production Games Inc,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,https://cdn.akamai.steamstatic.com/steam/apps/...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",0.0,"싱글 플레이어,추적되는 컨트롤러 지원,VR 전용","캐주얼,시뮬레이션,앞서 해보기",0.0,2021-10-11,Tigerlilly2931,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,https://cdn.akamai.steamstatic.com/steam/apps/...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,0.0,싱글 플레이어,"액션,인디,RPG",0.0,2021-05-23,Epic Banana Games LLC,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,https://cdn.akamai.steamstatic.com/steam/apps/...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,0.0,"멀티플레이어,MMO,PvP,온라인 PvP,협동,온라인 협동,플랫폼간 멀티플레이어,S...","액션,어드벤처,무료,대규모 멀티플레이어,RPG,시뮬레이션,전략",0.0,2021-05-09,Tactigon Games Studio,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,719890,Beasts of Bermuda,Beasts of Bermuda is a multiplayer game hostin...,2050000.0,"멀티플레이어,PvP,온라인 PvP","어드벤처,인디,대규모 멀티플레이어,시뮬레이션,앞서 해보기",5490.0,2018-12-21,"Sastrei Studios, LLC",0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,537000.0,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam 트레이딩 카드,S...","캐주얼,인디,RPG,전략",318.0,2020-03-13,Wonderbelly Games,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...
9864,548430,Deep Rock Galactic,"딥 락 갤럭틱은 1-4인의 플레이어가 협동하는 FPS 게임으로, 상남자 우주 드워프...",1023000.0,"싱글 플레이어,멀티플레이어,협동,온라인 협동,Steam 도전 과제,컨트롤러 완벽 지...",액션,155214.0,2020-05-13,Ghost Ship Games,85.0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,550000.0,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam Cloud,Ste...","액션,캐주얼,인디,RPG",0.0,2022-07-23,Cosmos,0.0,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,https://cdn.akamai.steamstatic.com/steam/apps/...


---

# price 전처리

In [35]:
# price에 00 떼기
def get_price(price):
    return price / 100

data['price'] = data['price'].apply(get_price)

---

# 형변환 (float -> int)

In [36]:
# float → int
data['price'] = data['price'].astype('int')
data['recommendations'] = data['recommendations'].astype('int')
data['metacritic'] = data['metacritic'].astype('int')

# str
data['release_date'] = data['release_date'].astype('str')
data['developers'] = data['developers'].astype('str')
data['screenshots'] = data['screenshots'].astype('str')

---

# Word2Vec를 위한 short_description 전처리
- 영어만 남기기.
- 모두 소문자로 변환.
- 문장부호 제거.

In [37]:
data['clean_description'] = data['short_description']

In [38]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np

In [39]:
def _removeNonAscii(s):
        return "".join(i for i in str(s) if  ord(i) < 128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [40]:
# Data cleaning
data['clean_description'] = data['clean_description'].apply(_removeNonAscii)
data['clean_description'] = data['clean_description'].apply(make_lower_case)
data['clean_description'] = data['clean_description'].apply(remove_stop_words)
data['clean_description'] = data['clean_description'].apply(remove_punctuation)

# 최종 확인

In [41]:
# 개수 : 65,683
data

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image,about_the_game,screenshots,clean_description
14,1609830,WillWalker,WillWalker is a frenetic shooter set in a cybe...,0,싱글 플레이어,"액션,캐주얼,무료,인디,전략",0,2021-05-24,DropOuts,0,https://cdn.akamai.steamstatic.com/steam/apps/...,WillWalker is a shooter game set in a cyberpun...,https://cdn.akamai.steamstatic.com/steam/apps/...,willwalker frenetic shooter set cyberpunk metr...
25,1610070,You Are Kaiju,Welcome to You are Kaiju. A rampage vr game wh...,0,"싱글 플레이어,추적되는 컨트롤러 지원,VR 전용","액션,앞서 해보기",0,2021-05-30,Production Games Inc,0,https://cdn.akamai.steamstatic.com/steam/apps/...,Welcome to You are Kaiju. This game involves y...,https://cdn.akamai.steamstatic.com/steam/apps/...,welcome kaiju rampage vr game player destroy c...
30,1610110,Valley of Dragons,"In Valley of Dragons, be prepared to raise and...",0,"싱글 플레이어,추적되는 컨트롤러 지원,VR 전용","캐주얼,시뮬레이션,앞서 해보기",0,2021-10-11,Tigerlilly2931,0,https://cdn.akamai.steamstatic.com/steam/apps/...,Love dragons? Want to raise a dragon of your o...,https://cdn.akamai.steamstatic.com/steam/apps/...,valley dragons prepared raise care personal dr...
36,1609390,DataJack,Take contracts from sinister megacorporations ...,0,싱글 플레이어,"액션,인디,RPG",0,2021-05-23,Epic Banana Games LLC,0,https://cdn.akamai.steamstatic.com/steam/apps/...,DataJack is a 2D isometric stealth action shoo...,https://cdn.akamai.steamstatic.com/steam/apps/...,take contracts sinister megacorporations isome...
39,1609590,World Senate,세계 상원 는 자신 만의 가상 생활을 할 수있는 온라인 브라우저 게임입니다. 당신은...,0,"멀티플레이어,MMO,PvP,온라인 PvP,협동,온라인 협동,플랫폼간 멀티플레이어,S...","액션,어드벤처,무료,대규모 멀티플레이어,RPG,시뮬레이션,전략",0,2021-05-09,Tactigon Games Studio,0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,719890,Beasts of Bermuda,Beasts of Bermuda is a multiplayer game hostin...,20500,"멀티플레이어,PvP,온라인 PvP","어드벤처,인디,대규모 멀티플레이어,시뮬레이션,앞서 해보기",5490,2018-12-21,"Sastrei Studios, LLC",0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,beasts bermuda multiplayer game hosting large ...
9862,848030,Roundguard,Roundguard는 핀볼처럼 튕겨나가면서 던전을 탐험하는 게임입니다. 다양한 전리...,5370,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam 트레이딩 카드,S...","캐주얼,인디,RPG,전략",318,2020-03-13,Wonderbelly Games,0,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,roundguard
9864,548430,Deep Rock Galactic,"딥 락 갤럭틱은 1-4인의 플레이어가 협동하는 FPS 게임으로, 상남자 우주 드워프...",10230,"싱글 플레이어,멀티플레이어,협동,온라인 협동,Steam 도전 과제,컨트롤러 완벽 지...",액션,155214,2020-05-13,Ghost Ship Games,85,https://cdn.akamai.steamstatic.com/steam/apps/...,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,fps
9871,2076850,Keyboard Warrior: Dreamstate,A game for people who love fast-paced action w...,5500,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam Cloud,Ste...","액션,캐주얼,인디,RPG",0,2022-07-23,Cosmos,0,https://cdn.akamai.steamstatic.com/steam/apps/...,***Something with my recording software funks ...,https://cdn.akamai.steamstatic.com/steam/apps/...,game people love fast paced action without fil...


# 최종 저장

In [42]:
data.to_excel(f'data/game_data_final.xlsx')